In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns             
import datetime
pd.set_option('display.max_rows', 500)
%matplotlib inline

In [2]:
# Source: http://web.mta.info/developers/turnstile.html
def get_data(week_nums):
    dfs = []
    for week_num in week_nums:
        file="/Users/poonamgaikar/metis/turnstile_"+str(week_num)+".txt"
        dfs.append(pd.read_csv(file))
    return pd.concat(dfs)
        
week_nums = [190518,190525,190601,190608,190615,190622,190629,190706,190713,190720,190727,190803]
df = get_data(week_nums)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/poonamgaikar/metis/turnstile_190518.txt'

In [ ]:
# Data cleaning

In [ ]:
df.columns = df.columns.str.replace(' ', '') #removed spaces rename column

In [7]:
station_mask = (
                (df["STATION"] == "ATLANTIC AV") |
                (df["STATION"] == "FLATBUSH AV-B.C")|
                (df["STATION"] == "ATL AV-BARCLAY")|
                (df["STATION"] == "25 AV")|
                (df["STATION"] == "BERGEN ST")|
                (df["STATION"] == "FULTON ST")|
                (df["STATION"] == "LAFAYETTE AV")
               )
station_mask

NameError: name 'df' is not defined

In [ ]:
ss=df[station_mask]

In [6]:
ss=ss.drop(columns=['LINENAME','DIVISION','DESC']) #droped unwated columnsz

NameError: name 'ss' is not defined

In [3]:
ss["TURNSTILE"]=ss[ "C/A"]+ss["UNIT"]+ss["SCP"]+ss["STATION"]

NameError: name 'ss' is not defined

In [4]:
ss[(ss["TURNSTILE"] =="N098R02800-02-01FULTON ST") &
    ((ss["DATE"] == "07/28/2021") |
     (ss["DATE"] == "07/27/2021"))]

NameError: name 'ss' is not defined

In [5]:
ss = (ss.groupby(
    ["TURNSTILE","DATE","STATION"],
    as_index=False).ENTRIES.first())

NameError: name 'ss' is not defined

In [ ]:
ss[["PREV_DATE", "PREV_ENTRIES"]] = (ss.groupby(["TURNSTILE"])["DATE", "ENTRIES"].apply(lambda grp: grp.shift(1)))

In [ ]:
ss[(ss["TURNSTILE"] =="N098R02800-02-01FULTON ST") &
    ((ss["DATE"] == "07/28/2021") |
     (ss["DATE"] == "07/27/2021"))]

In [ ]:
# Drop the rows for the earliest date in the df
ss.dropna(subset=["PREV_DATE"], axis=0, inplace=True)

In [ ]:
def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        counter = -counter
    if counter > max_counter:
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        return 0
    if counter > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        return 0
    return counter

# If counter is > 1Million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits


In [ ]:
ss["DAILY_ENTRIES"]= ss.apply(get_daily_counts, axis=1, max_counter=100000)

In [ ]:
ss.head(10)

In [ ]:
ss["DATE_TIME"]=pd.to_datetime(ss["DATE"])

In [ ]:
ss["WEEK"]=ss["DATE_TIME"].dt.isocalendar().week

In [ ]:
ss["DAY"]=ss["DATE_TIME"].dt.dayofweek

In [ ]:
MASK=((ss["DAY"] == 0) |
      (ss["DAY"] == 1) |
      (ss["DAY"] == 2) |
      (ss["DAY"] == 3) |
      (ss["DAY"] == 4) 
     )
      

In [ ]:
ss=ss[MASK]

In [ ]:
t=ss[( (ss["WEEK"]==30) &
     (ss["STATION"] == "FULTON ST" )  
   )]

In [ ]:
t.sort_values(by="DAILY_ENTRIES", ascending=False)

In [ ]:
VOLUME_PER_STATION=ss.groupby(["WEEK","STATION"])["DAILY_ENTRIES"].sum().reset_index()

In [ ]:
VOLUME_PER_STATION

In [ ]:
VOLUME_PER_WEEK=ss.groupby(["WEEK"])["DAILY_ENTRIES"].sum().reset_index()

In [ ]:
VOLUME_PER_WEEK.head(50)

In [ ]:
plt.figure(figsize=(20,5))
# plt.fill_between(WEEKLY_ENTRIES['WEEK'], WEEKLY_ENTRIES['ENTRIES'], color='skyblue', alpha=0.5)
plt.bar(VOLUME_PER_WEEK['WEEK'], VOLUME_PER_WEEK['DAILY_ENTRIES'], color="Slateblue", alpha=0.6)
plt.xlabel('WEEK')
plt.ylabel('ENTRIES')
plt.ylim(0,1000000)
plt.title('Total Number of Entries')
plt.locator_params(axis='x', nbins=40)
plt.ticklabel_format(axis="y", style="plain");

In [ ]:
plt.figure(figsize=(10,5))
# plt.fill_between(WEEKLY_ENTRIES['WEEK'], WEEKLY_ENTRIES['ENTRIES'], color='skyblue', alpha=0.5)
plt.plot(VOLUME_PER_STATION['WEEK'], VOLUME_PER_STATION['DAILY_VOLUME'], color="Slateblue", alpha=0.6)
plt.xlabel('WEEK')
plt.ylabel('ENTRIES')
plt.ylim(0,1600000)
plt.title('Total Number of Entries')
plt.locator_params(axis='x', nbins=26)
plt.ticklabel_format(axis="y", style="plain");

In [ ]:
plt.figure(figsize=(14, 8))
plt.title("Total Traffic For Stations Near Barclays center")
graph1DF = VOLUME_PER_STATION.groupby(['STATION']).DAILY_ENTRIES.agg(['sum']).reset_index()
graph1DF = graph1DF.nlargest(10,'sum')
sns.barplot(x="DAILY_ENTRIES", y="STATION", orient = "h", data=VOLUME_PER_STATION, palette = "Blues_d")
plt.savefig('Stations near Barclays.png', dpi = 1000)